1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변 생성 못 할수 있음
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [18]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("tax_with_markdown.docx")

document_list = loader.load_and_split(text_splitter)

In [19]:
len(document_list)


225

In [20]:
document_list[52]

Document(metadata={'source': 'tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
import os
from langchain_upstage import UpstageEmbeddings
embedding = UpstageEmbeddings(model="embedding-query")

In [23]:
import os

from pinecone import Pinecone

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


In [24]:
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [37]:
query = '연봉 5천만원인 직장인의 소득세는?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [38]:
retrieved_docs

[Document(id='bfa37203-112f-465c-b115-7b171345ae05', metadata={'source': 'tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세

In [39]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-mini")

In [40]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# [Question]
# {query}
# """

In [41]:
# ai_message = llm.invoke(prompt)

In [42]:
# ai_message.content

In [43]:
from langchain import hub
from langchain.chains import RetrievalQA

prompt = hub.pull("rlm/rag-prompt")

/opt/anaconda3/envs/project1122env/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [44]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [45]:
ai_message = qa_chain({"query": query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는?',
 'result': '연봉 5천만원인 직장인의 소득세는 종합소득 과세표준 5,000만원 초과 8,800만원 이하에 해당하여, 624만원 + (5,000만원을 초과하는 금액의 24%)를 계산하면 됩니다.'}

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()


In [51]:
new_question = dictionary_chain.invoke({"question": query})
print(query)
print(new_question)

연봉 5천만원인 직장인의 소득세는?
소득세율이 5천만원인 거주자의 소득세는 얼마인가요?


In [52]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [53]:
ai_response = tax_chain.invoke({"question": query})
ai_response

{'query': '거주자의 연봉이 5천만원일 때, 소득세는 어떻게 되나요?',
 'result': '거주자의 연봉이 5천만원일 경우, 소득세율은 5,000만원 초과 8,800만원 이하에 해당합니다. 따라서, 소득세율은 24%이며, 계산식은 종합소득 과세표준에 세율을 적용하여 계산한 금액이 됩니다.'}